# Get Radii of Gyration for Outputs

#### USER INPUTS

Specify the output directory for which radii of gyration are to be calculated.

In [1]:
out_dir = "/scratch/users/jwakim/sliding_nucleosome/output/final_sims"

Specify the resources to request to each job.

In [2]:
runtime = "2:59:59"
num_tasks = "1"
cpus_per_task = "1"
mem_per_cpu = "4G"

#### Import Modules

In [3]:
import os
import sys
from datetime import datetime
import numpy as np

#### List Simulations in the Output Directory

In [4]:
# List the simulation directories
sims = os.listdir(out_dir)
sims = [sim for sim in sims if os.path.isdir(os.path.join(out_dir, sim)) and sim.startswith("sim_")]

# Sort the simulation directories by their indices
sim_inds = [int(sim.split("_")[-1]) for sim in sims]
sims = [sim for _, sim in sorted(zip(sim_inds, sims))]
sim_inds = np.sort(sim_inds)

#### Navigate to Root Directory

In [5]:
# Get the absolute path of the notebook's directory
notebook_directory = os.path.dirname(os.path.abspath('__file__'))

# Navigate one level up to get the root directory path
root_directory = os.path.abspath(os.path.join(notebook_directory, '..'))

# Change the working directory to the root directory
os.chdir(root_directory)

# Append the root directory to sys.path
sys.path.append(root_directory)

#### Generate Directory for SBATCH files

In [6]:
# Generate a directory for ALL sets of sbatch files (if it does not exist)
sbatch_dir = os.path.join(root_directory, "sbatch_files")
if not os.path.exists(sbatch_dir):
    os.makedirs(sbatch_dir)

In [7]:
# Generate a subdirectory for the CURRENT set of sbatch files
now = datetime.now()
subdirectory_name = now.strftime("find_Rg_%Y-%m-%d_%H-%M-%S")
slurm_dir = os.path.join(sbatch_dir, subdirectory_name)
os.makedirs(slurm_dir)

#### Write the SBATCH files

In [8]:
analysis_dir = f"{root_directory}/examples"
analysis_file = "get_Rg.py"
slurm_prefix = "find_Rg"
run_file_prefix = "run_jobs"
job_prefix = "find_Rg"

In [9]:
job_count = 0
batch_size = 900
n_batches = int(np.ceil(len(sims) / batch_size))
run_files = {i: [] for i in range(n_batches)}

for i, sim in enumerate(sims):
    batch_ind = int(job_count // batch_size)
    sim_path = os.path.join(out_dir, sim)
    slurm_file_name = f"{slurm_prefix}_{job_count}.slurm"
    job_name = f"{job_prefix}_{job_count}"
    run_files[batch_ind].append(slurm_file_name)
    slurm_file_path = os.path.join(slurm_dir, slurm_file_name)

    with open(slurm_file_path, 'w') as f:

        f.write("#!/bin/bash\n\n")

        f.write(f"#SBATCH --job-name={job_name}\n")
        f.write(f"#SBATCH --time={runtime}\n")
        f.write(f"#SBATCH --ntasks={num_tasks}\n")
        f.write(f"#SBATCH --cpus-per-task={cpus_per_task}\n")
        f.write(f"#SBATCH --mem-per-cpu={mem_per_cpu}\n\n")

        f.write(f"source ~/.bashrc\n")
        f.write(f"conda activate slide\n")
        f.write(f"cd {analysis_dir}\n")
        f.write(f"python {analysis_file} {os.path.join(out_dir, sim)}\n")
        f.write(f"echo 'Job complete!'\n")

    job_count += 1

In [10]:
for i in range(n_batches):
    run_file_name = f"{run_file_prefix}_{i}.sh"
    with open(os.path.join(slurm_dir, run_file_name), "w") as f:
        for job_file in run_files[i]:
            f.write(f"sbatch {job_file}\n")